In [1]:
import os
import pytorch_lightning as pl

import glob
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import KFold, StratifiedKFold

# from data_loader import CustomDataset
from sklearn import preprocessing

In [2]:
all_img_list = glob.glob('../aug_data/train/*/*')
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : str(x).split('/')[-2])

kf = StratifiedKFold(n_splits=5,
            shuffle=True,
            random_state=41)

all_splits = [k for k in kf.split(df, df['label'])]
train_idx, val_idx = all_splits[0]
train_idx, val_idx = train_idx.tolist(), val_idx.tolist()

In [14]:
from sklearn.metrics import classification_report

y_true = [0, 1, 2, 0, 1, 2, 3]
y_pred = [0, 2, 1, 0, 2, 1, 1]

target_names = ['class 0', 'class 1', 'class 2', 'class 3']
# target_names = [f'cls {idx}' for idx in range(19)]
print(target_names)

print(classification_report(y_true, y_pred, target_names=target_names))

['class 0', 'class 1', 'class 2', 'class 3']
              precision    recall  f1-score   support

     class 0       1.00      1.00      1.00         2
     class 1       0.00      0.00      0.00         2
     class 2       0.00      0.00      0.00         2
     class 3       0.00      0.00      0.00         1

    accuracy                           0.29         7
   macro avg       0.25      0.25      0.25         7
weighted avg       0.29      0.29      0.29         7



/opt/conda/envs/paper/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/paper/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/paper/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [36]:
import re
from sklearn.metrics import classification_report

y_true = [0, 1, 2, 0, 1, 2, 0]
y_pred = [0, 2, 1, 0, 2, 1, 1]

target_names = ['class0', 'class1', 'class2']

report = classification_report(y_true, y_pred, target_names=target_names)
report_lines = report.split('\n')

# 클래스별 지표 추출
metrics = {}
print(report_lines)
print(len(report_lines))
print()
for line in report_lines:
    print(f'line content: {line}')
    if not line and len(line) > 2:
        class_name, *class_metrics, support = line.split()
        print(f'cls name: {class_name}')
        print(f'class_metrics: {class_metrics}')
        print(f'support {support}')
    # metrics[class_name] = {
    #     'precision': float(class_metrics[0]),
    #     'recall': float(class_metrics[1]),
    #     'f1-score': float(class_metrics[2]),
    #     'support': int(support),
    # }
print('done')

['              precision    recall  f1-score   support', '', '      class0       1.00      0.67      0.80         3', '      class1       0.00      0.00      0.00         2', '      class2       0.00      0.00      0.00         2', '', '    accuracy                           0.29         7', '   macro avg       0.33      0.22      0.27         7', 'weighted avg       0.43      0.29      0.34         7', '']
10

line content:               precision    recall  f1-score   support
line content: 
line content:       class0       1.00      0.67      0.80         3
line content:       class1       0.00      0.00      0.00         2
line content:       class2       0.00      0.00      0.00         2
line content: 
line content:     accuracy                           0.29         7
line content:    macro avg       0.33      0.22      0.27         7
line content: weighted avg       0.43      0.29      0.34         7
line content: 
done


In [5]:
print(f'all splits length: [{len(all_splits)}]')
print(f'all splits[0][0] shape: [{all_splits[0][0].shape}]') # 80%
print(f'all splits[0][1] shape: [{all_splits[0][1].shape}]') # 20%

all splits length: [5]
all splits[0][0] shape: [(2896,)]
all splits[0][1] shape: [(724,)]


In [6]:
print(f'df shape from train_idx : [{df.iloc[train_idx].shape}]')
print(f'df shape from val_idx : [{df.iloc[val_idx].shape}]')

train = df.iloc[train_idx]
val = df.iloc[val_idx]

df shape from train_idx : [(2896, 2)]
df shape from val_idx : [(724, 2)]


In [10]:
le = preprocessing.LabelEncoder()
df['label'] = le.fit_transform(df['label'])
df

,img_path,label
0,../aug_data/train/들뜸/24.png,5
1,../aug_data/train/들뜸/34.png,5
2,../aug_data/train/들뜸/25.png,5
3,../aug_data/train/들뜸/5.png,5
4,../aug_data/train/들뜸/8.png,5
...,...,...
3615,../aug_data/train/틈새과다/0.png,16
3616,../aug_data/train/틈새과다/3.png,16
3617,../aug_data/train/틈새과다/4.png,16
3618,../aug_data/train/틈새과다/1.png,16


In [24]:
import torch

torch.Tensor(df.label.value_counts().to_frame().sort_index().values).squeeze()

tensor([  12.,  317.,  187.,  219.,   21.,   57.,  107.,  134.,   28.,   60.,
         587.,  149.,   54.,   27.,   34.,  192.,    5.,   55., 1375.])

In [26]:
LADELoss(19, df.label.value_counts().to_frame().sort_index().values)

LADELoss()

In [25]:

import functools

import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
# from utils import *


class LADELoss(nn.Module):
    def __init__(self, num_classes=10, prior=None, remine_lambda=0.1):
        super().__init__()
        if prior is not None:
            self.img_num_per_cls = torch.Tensor(prior).squeeze().cuda()
            self.prior = self.img_num_per_cls / self.img_num_per_cls.sum()
        else:
            self.prior = None

        self.balanced_prior = torch.tensor(1. / num_classes).float().cuda()
        self.remine_lambda = remine_lambda

        self.num_classes = num_classes
        self.cls_weight = (self.img_num_per_cls.float() / torch.sum(self.img_num_per_cls.float())).cuda()

    def mine_lower_bound(self, x_p, x_q, num_samples_per_cls):
        N = x_p.size(-1)
        first_term = torch.sum(x_p, -1) / (num_samples_per_cls + 1e-8)
        second_term = torch.logsumexp(x_q, -1) - np.log(N)

        return first_term - second_term, first_term, second_term

    def remine_lower_bound(self, x_p, x_q, num_samples_per_cls):
        loss, first_term, second_term = self.mine_lower_bound(x_p, x_q, num_samples_per_cls)
        reg = (second_term ** 2) * self.remine_lambda
        return loss - reg, first_term, second_term

    def forward(self, y_pred, target, q_pred=None):
        """
        y_pred: N x C
        target: N
        """
        per_cls_pred_spread = y_pred.T * (target == torch.arange(0, self.num_classes).view(-1, 1).type_as(target))  # C x N
        pred_spread = (y_pred - torch.log(self.prior + 1e-9) + torch.log(self.balanced_prior + 1e-9)).T  # C x N

        num_samples_per_cls = torch.sum(target == torch.arange(0, self.num_classes).view(-1, 1).type_as(target), -1).float()  # C
        estim_loss, first_term, second_term = self.remine_lower_bound(per_cls_pred_spread, pred_spread, num_samples_per_cls)

        loss = -torch.sum(estim_loss * self.cls_weight)
        return loss

In [7]:
train.label.value_counts()

18    1100
10     469
1      253
3      175
15     153
2      150
11     119
7      108
6       86
9       48
5       46
17      44
12      43
14      28
13      22
8       22
4       17
0        9
16       4
Name: label, dtype: int64

In [8]:
val.label.value_counts()

18    275
10    118
1      64
3      44
15     39
2      37
11     30
7      26
6      21
9      12
5      11
12     11
17     11
8       6
14      6
13      5
4       4
0       3
16      1
Name: label, dtype: int64

In [ ]:
train_dataset = CustomDataset(train['img_path'].values, train['label'].values, train_transform_4_origin)